# 12 用 LangChain 實作新書 宣傳自動小編

## 12-2 熟悉 LangChain 基本功能

**設定環境**

安裝 LangChain、openai、python-dotenv 套件

In [ ]:
!pip install langchain
!pip install openai
!pip install python-dotenv

設定環境變數

In [ ]:
from dotenv import load_dotenv
load_dotenv()

使用 ChatOpenAI 模組

In [ ]:
# 匯入 langchain 模組
from langchain.chat_models import ChatOpenAI

### 使用 ChatOpenAI

用HumanMessage表示使用者的訊息,SystemMessage表示AI的角色, AIMessage表示AI生成的回覆。

In [ ]:
# 匯入相關類別
from langchain.schema import (
        AIMessage,
        HumanMessage,
        SystemMessage
)

In [ ]:
# ChatOpenAI 聊天模型
model = ChatOpenAI(temperature=0) # 預設模型為 "gpt-3.5-turbo"
# 使用 HumanMessage 輸入要求
messages = ([HumanMessage(
             content="回答時使用繁體中文和台灣詞語解釋,"
                     "問題：NBA 的英文全名是？")])
response = model(messages)
# 輸出以 AIMessage 的形式呈現
print(response.content)

In [ ]:
model.predict("回答時使用繁體中文和台灣詞語解釋,"
              "問題:NBA 的英文全名是？")

### 提示模板 (PromptTemplate)

In [ ]:
# 匯入相關類別
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
# 給AI角色和要做的事情, 並設參數
system_message = "你是一位助手, 回答時答案使用 {input_language}"
human_message = "問題：{text}"

In [ ]:
# 參數化 prompt
system_template = SystemMessagePromptTemplate.from_template(
    system_message)
human_template = HumanMessagePromptTemplate.from_template(
    human_message)

In [ ]:
# 結合 system_message_prompt、human_message_prompt 組成對話提示模板
chat_prompt = ChatPromptTemplate.from_messages(
    [system_template,human_template])

輸入參數丟回給模型

In [ ]:
# 將參數傳回到提示模板, 變成 user、system 形式
messages = chat_prompt.format_messages(
    input_language="繁體中文和台灣詞語",
    text="台灣最高的建築物是？")
# 將 Chat 形式傳回給模型
reponse = model(messages)
print(reponse.content)

### 建立 LLMChain

In [ ]:
from langchain import LLMChain

使用 LLMChain 串接模型和提示模板

In [ ]:
chain = LLMChain(llm=model, prompt=chat_prompt)
answer = chain.run(input_language="繁體中文和台灣詞語",
                      text="台灣最高的建築物是？")
print(answer)

可以更改輸入做嘗試

### 對話記憶 Memory

記錄對話內容

In [ ]:
# 匯入類別
from langchain.prompts import MessagesPlaceholder
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
ai_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "以下是使用者與 AI 之間友好的對話,"
        "AI 很健談且能根據上下文提供具體細節,"
        "如果 AI 不知道問題的答案會如實說不知道"
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

In [ ]:
# ConversationBufferMemory 記錄對話內容
memory = ConversationBufferMemory(return_messages=True)
# ConversationChain 專門使用在對話聊天上
conversation = ConversationChain(memory=memory,
                                 prompt=ai_prompt,
                                 llm=model,
                                 verbose=True)
conversation.predict(input="嗨, 你好")

In [ ]:
conversation.predict(input="可以介紹你自己嗎？")

加入迴圈變成 ChatGPT

In [ ]:
while True:
    # 輸入問題
    question = input("請輸入問題：")
    # 不輸入時按 Enter 退出程式
    if not question.strip(): break
    reponse=conversation.predict(input=question)
    print(reponse)

## 12-3 串接 Google search 及 Agent 運用

### 建立 LangChain 代理 (Agent)

請在 .env 檔中新增環境變數 GOOGLE_CSE_ID 和 GOOGLE_API_KEY 後重新上傳到 Colab 中

In [ ]:
# 再次使用 dotenv 建立環境變數
load_dotenv()

In [ ]:
# 匯入類別
from langchain import GoogleSearchAPIWrapper
from langchain import LLMMathChain

In [ ]:
# 建立物件
search = GoogleSearchAPIWrapper()

在本機套用 GoogleSearchAPIWrapper 必須先 pip install google-api-python-client

Colab 本身已經預先安裝好了

In [ ]:
# pip install google-api-python-client

LangChain 裡面專門計算的工具

In [ ]:
llm_math_chain = LLMMathChain.from_llm(llm=model,verbose=True)

### 建立工具組

使用 Tool 將函數分別定義, 並用寫進 list 中

In [ ]:
from langchain.agents import Tool

In [ ]:
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="如果不曉得答案, 用此工具搜尋資料"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="如果需要數學計算, 使用此工具"
    )]

### 建立 Agent

initialize_agent 初始化代理

In [ ]:
from langchain.agents import initialize_agent, AgentType

In [ ]:
agent = initialize_agent(llm=model,
            tools=tools,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True)

使用 Google Search 和 Calculator 工具

In [ ]:
reponse = agent.run("台灣最高的建築物相當於世界上最高的建築物的幾倍？")
# 有時會失敗多按幾次就會成功
print(reponse)

## 12-4 實作 用 LangChain 在社交軟體上宣傳新書

### 建立 Chain 元件

In [ ]:
book_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
    "你是一名FB小編，任何新書摘要都能重新改寫成新書宣傳文,"
    "且有FB標籤和圖案點綴,不需要圖片和連結,在2000個tokens以內,輸出為完整文案:"),
    HumanMessagePromptTemplate.from_template(
    "書名:{book_name}, 摘要:{book_content}")])
llm_chain = LLMChain(llm=model, prompt=book_prompt)

使用 Memory 保留對話紀錄,方便修改文案

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

安裝 tiktioken 套件

In [ ]:
!pip install tiktoken

In [ ]:
ai_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
    "你是一名FB小編, 會根據上下文判斷使用者的要求,"
    "修改宣傳文案內容且不會回覆感謝詞以及以下更改內容, 回答請使用繁體中文"),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])
memory = ConversationSummaryBufferMemory(llm=model,
                                         return_messages=True,
                                         max_token_limit=2000)
book_modify = ConversationChain(memory=memory,
                                prompt=ai_prompt,
                                llm=model)

### 使用爬蟲

安裝爬蟲模組

In [ ]:
!pip install requests
!pip install bs4
!pip install lxml

In [ ]:
# 匯入模組
import requests
from bs4 import BeautifulSoup

### 建立可取得單一書籍文案的函式 flag_book

In [ ]:
def flag_book(code):
    memory.clear()
    # 公司新書網址
    url = f'https://www.flag.com.tw/books/product/{code}'
    product_response = requests.get(url)
    product_content = product_response.text
    # 取得 HTML
    product_soup = BeautifulSoup(product_content, "lxml")
    # 提取書名
    book_name = product_soup.find("h1").text.strip()
    # 提取簡介
    description = product_soup.find(
        "p", class_="atext").text.strip()
    # 生成宣傳文
    answer = llm_chain.run(book_name=book_name,
                           book_content=description)
    # 丟入修改元件中
    book_modify.predict(input=answer)
    # 回傳到主程式
    return answer

### 宣傳文修改程式

手動修改程式

In [ ]:
def modify(query):
    # flag 為限制器
    flag = 0
    while True:
        # 輸入要修改的地方
        question = input("請輸入要修改的地方：")
        # 不輸入時可以退出修改程序,但因為要讓 AI 生成一次,所以 flag=1 才能退出
        if not question.strip()  and flag == 0:
            print("尚未進行更改，至少要讓 AI 生成一次才能退出程序")
            continue
        elif not question.strip() and flag == 1:
            ai_response = book_modify.predict(input="呈現完整文案")
            return ai_response
        # 將問題丟入到 Chain 中，AI 根據上下文回答
        ai_response = book_modify.predict(input=question)
        print(ai_response)
        flag = 1

自動修改程式

In [ ]:
def auto_modify(query):
    book_modify.predict(input=query)
    ai_response = book_modify.predict(input="呈現完整文案")
    return ai_response

### 前往 IFTTT 連接 FB

先建立FB粉絲專頁和IFTTT設定

In [ ]:
# 因為又新增了一個 IFTTT_KEY,所以要再設定一次環境變數
import os
load_dotenv()
ifttt_url = os.getenv('IFTTT_URL')

In [ ]:
# 匯入 IFTTT 類別
from langchain.tools.ifttt import IFTTTWebhook

以下程式執行完畢請到FB粉絲專業確認貼文上傳情形

### 宣傳文編碼與上傳

In [ ]:
def ifttt_tool(query):
    code, text = query.split(",")
    text = text.replace('#','%23')
    tool = IFTTTWebhook(
        name = "Facebook",
        description = "Create a link post on Flag_new_book",
        url = (f"{ifttt_url}?value1="
               f"https://www.flag.com.tw/books/product/{code}"
               f"\&value2={text}"))
    return tool.run("上傳")

In [ ]:
tools = [Tool(
         name = "Book",
         func = flag_book,
         description = "此工具負責生成宣傳文內容, 輸入為代號"),

         Tool(
         name = "Manual_modify",
         func = modify,
         description = "手動修改宣傳文內容時使用這工具, 輸入為宣傳文內容"),

         Tool(
         name = "Auto_modify",
         func = auto_modify,
         description = "自動修改宣傳文內容使用這工具, 輸入為修改要求"),

         Tool(
         name = "Upload",
         func = ifttt_tool,
         description = "上傳宣傳文內容使用這工具, 輸入為代號和不是字串的宣傳文內容",
         return_direct=True)]

In [ ]:
agent= initialize_agent(llm=model,
                    tools=tools,
                    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                    verbose=True)

In [ ]:
agent.run(input="幫我把書號 F3165 做成宣傳文, 修改後上傳")

In [ ]:
agent.run(input="幫我把書號 F3165 做成宣傳文,"
                "並自動修改為'在最後一行增加我愛新書'後上傳")